In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import fsspec
import huggingface_hub

In [2]:
pip install huggingface_hub

  Using cached huggingface_hub-0.30.1-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached huggingface_hub-0.30.1-py3-none-any.whl (481 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install ipywidgets --upgrade


In [ ]:
pip install jupyter --upgrade


In [24]:
#pip install pyarrow


SyntaxError: invalid syntax (787043912.py, line 2)

In [25]:
#pip install fsspec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.2/481.2 kB 9.7 MB/s eta 0:00:0010.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 10.4 MB/s eta 0:00:000:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Read only the first 100 rows of the parquet file
df = pd.read_parquet("data/food_fixed.parquet", engine='pyarrow').head(100)
print(df)

In [1]:
df.head()

NameError: name 'df' is not defined

In [1]:
import pyarrow.parquet as pq

In [2]:
# Open the Parquet file
table = pq.read_metadata("data/food_fixed.parquet")

In [3]:
# Get the schema (column names, types)
print(table.schema)

required group field_id=-1 schema {
  optional int32 field_id=-1 additives_n;
  optional group field_id=-1 additives_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional group field_id=-1 allergens_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional group field_id=-1 brands_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional binary field_id=-1 brands (String);
  optional binary field_id=-1 categories (String);
  optional group field_id=-1 categories_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional group field_id=-1 checkers_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional group field_id=-1 ciqual_food_name_tags (List) {
  

In [18]:
def rename_columns_in_schema(schema):
    """Recursively rename fields in the schema, including nested structures like ListType and StructType."""
    def rename_field(field):
        # Check if the field name starts with a digit, and if so, rename it by adding an underscore at the beginning
        if str(field.name)[0].isdigit():
            field = field.clone(name=f"_{field.name}")
        
        # If the field is a list (ListType), recursively rename the item inside the list
        if isinstance(field.type, pa.ListType):
            # Recursively call rename_field on the value type of the list
            field = field.clone(type=pa.list(rename_field(field.type.value_type)))
        
        # If the field is a struct (StructType), recursively rename each field inside the struct
        elif isinstance(field.type, pa.StructType):
            field = field.clone(type=pa.struct([rename_field(f) for f in field.type]))
        
        return field

    # Apply the renaming function to each field in the schema
    return pa.schema([rename_field(field) for field in schema])

def fix_column_names_streaming(parquet_file, output_file):
    """Fix column names in a Parquet file without loading the entire file into memory."""
    # Read Parquet metadata and schema
    reader = pq.ParquetFile(parquet_file)  # Read Parquet metadata
    schema = reader.schema

    # Rename columns in the schema
    updated_schema = rename_columns_in_schema(schema)

    # Now write the Parquet file with the updated schema
    with pq.ParquetWriter(output_file, schema=updated_schema, use_dictionary=True) as writer:
        # Process and write data in batches to avoid loading entire file into memory
        for batch in reader.iter_batches(batch_size=100_000):  # Process in chunks
            # Rename columns for this batch of data
            table = pa.Table.from_batches([batch])
            writer.write_table(table.rename_columns(updated_schema.names))  # Rename and write

    print(f"Fixed column names. New file saved as: {output_file}")

In [20]:
fix_column_names_streaming("data/food_fixed.parquet","data/food_fixed2.parquet")

AttributeError: 'pyarrow._parquet.ColumnSchema' object has no attribute 'type'

In [16]:
# Open the Parquet file
table = pq.read_metadata("data/food_fixed1.parquet")
# Get the schema (column names, types)
print(table.schema)

required group field_id=-1 schema {
  optional int32 field_id=-1 additives_n;
  optional group field_id=-1 additives_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional group field_id=-1 allergens_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional group field_id=-1 brands_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional binary field_id=-1 brands (String);
  optional binary field_id=-1 categories (String);
  optional group field_id=-1 categories_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional group field_id=-1 checkers_tags (List) {
    repeated group field_id=-1 list {
      optional binary field_id=-1 element (String);
    }
  }
  optional group field_id=-1 ciqual_food_name_tags (List) {
  

In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'food': 'food.parquet', 'beauty': 'beauty.parquet'}
# Read the large Parquet file
df = pd.read_parquet("hf://datasets/openfoodfacts/product-database/" + splits["food"])

# Split data and save smaller files
categories = df['category'].unique()  # Assume you have a 'category' column
for category in categories:
    df_category = df[df['category'] == category]
    df_category.to_parquet(f"food_{category}.parquet")

In [3]:
import pyarrow.parquet as pq

# Define the file path
file_path = "hf://datasets/openfoodfacts/product-database/food.parquet"

# Open the Parquet file
parquet_file = pq.ParquetFile(file_path)

# Iterate over the file in batches
batch_size = 100000  # Number of rows per batch
for batch in parquet_file.iter_batches(batch_size=batch_size):
    # Convert batch to Pandas DataFrame
    df_chunk = batch.to_pandas()
    print(df_chunk.shape)  # Process each chunk here
    # Add your processing logic

ArrowInvalid: Unrecognized filesystem type in URI: hf://datasets/openfoodfacts/product-database/food.parquet

In [5]:
pip install pyspark


  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session with increased memory settings
spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.ui.port", "4040") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "8g") \
    .appName("MySparkApp") \
    .config("spark.executor.cores", "4") \
    .config("spark.num.executors", "4") \
    .getOrCreate()


df = spark.read.parquet("data/food.parquet")



25/04/01 21:41:55 WARN Utils: Your hostname, DESKTOP-QRKC0LG resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/01 21:41:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 21:41:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

In [2]:
df.show(2)

25/04/01 21:37:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+-----------+--------------+--------------+------------+-------+--------------------+--------------------+-------------+---------------------+-----------+-------------+--------------------+--------+------------+--------------------+------------------+----------+---------------+------------------------+----------------------+--------------------------+--------------------+-------------+--------------+--------------+-------------+-------+--------------+---------+--------------------+--------------------+------------+--------------------+--------------------+-------------------------+---------------------------+-------------+-------------------------+----------------------------+--------------------+--------------------+------------------------------------+--------------------------------------+----------------------------------+--------------------------------+--------------------+-------------------+--------------------+--------------------+----+------------------+--------------------+-

In [3]:
# Filter the rows where 'categories' contains 'baby food' (case insensitive)
filtered_df = df.filter(df['categories'].rlike('(?i).*baby food.*'))

# Show the result
filtered_df.show(2)


[Stage 3:>                                                          (0 + 1) / 1]

+-----------+-------------------+--------------+--------------+------------+--------------------+--------------------+-------------+---------------------+-----------+-------------+--------------------+--------+------------+--------------------+------------------+----------+---------------+------------------------+----------------------+--------------------------+--------------------+--------------------+--------------+--------------+----------------+-------+--------------+---------+--------------------+--------------------+------------+--------------------+--------------------+-------------------------+---------------------------+-------------+-------------------------+----------------------------+--------------------+--------------------+------------------------------------+--------------------------------------+----------------------------------+--------------------------------+--------------------+-------------------+--------------------+--------------------+----+------------------+

In [2]:
# Get the column names from the DataFrame
column_names = df.columns




In [3]:
# Check which columns start with a digit
columns_starting_with_digit = [col for col in column_names if col[0].isdigit()]

# Print the columns that start with a digit
if columns_starting_with_digit:
    print(f"Columns starting with a digit: {columns_starting_with_digit}")
else:
    print("No columns start with a digit.")

No columns start with a digit.


In [4]:
# List of columns to select
selected_columns = [
    "additives_n", "additives_tags", "allergens_tags", "brands_tags", "brands",
    "categories", "categories_tags", "created_t", "creator", "generic_name"
]

# Select the columns from the DataFrame
selected_df = df.select(*selected_columns)
# Show the first 5 rows
selected_df.show(5)


+-----------+--------------+--------------+------------+-------+--------------------+--------------------+----------+---------------+------------+
|additives_n|additives_tags|allergens_tags| brands_tags| brands|          categories|     categories_tags| created_t|        creator|generic_name|
+-----------+--------------+--------------+------------+-------+--------------------+--------------------+----------+---------------+------------+
|       NULL|          NULL|     [en:nuts]|[xx:bovetti]|Bovetti|Petit-déjeuners,P...|[en:breakfasts, e...|1519297017|        kiliweb|          []|
|          0|            []|            []|    [lagg-s]| Lagg's|                null|           [en:null]|1489055330|usda-ndb-import|          []|
|          0|            []|            []|    [lagg-s]| Lagg's|Plant-based foods...|[en:plant-based-f...|1489055331|usda-ndb-import|          []|
|          0|            []|            []|    [lagg-s]| Lagg's|                null|           [en:null]|1489055331|u

In [5]:
# Select the first 10 rows
df_limit_10 = df.limit(10)

# Convert to Pandas DataFrame
pandas_df = df_limit_10.toPandas()

# Write to CSV file
pandas_df.to_csv("output_file.csv", index=False)

25/04/01 21:46:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                